In [18]:

import tensorflow as tf
from pathlib import Path
import pandas as pd
import os


In [5]:
try:
    import google.colab
    model_dir = Path('/content/drive/My Drive/Colab_Notebooks/state_farm/models') # for colab
    train_img_dir = '/content/drive/My Drive/Colab_Notebooks/state_farm/data/imgs/train'
    test_img_dir = Path('/content/drive/My Drive/Colab_Notebooks/state_farm/data/imgs/test')
    csv_dir = '/content/drive/My Drive/Colab_Notebooks/state_farm/data/csv'
    csv_file = 'driver_imgs_list.csv'
    submissions_dir = '/content/drive/My Drive/Colab_Notebooks/state_farm/submissions'
    code_in = 'Google Colab'
    drive.mount('/content/drive')
except:
    model_dir = Path('../models') # for local machine
    train_img_dir = '../data/imgs/train'
    test_img_dir = Path('../data/imgs/test')
    csv_dir = '../data/csv'
    csv_file = 'driver_imgs_list.csv'
    submissions_dir = '../submissions'
    code_in = 'Local Machine'

print(f'Code is run from {code_in}')

Code is run from Local Machine


In [6]:
class_labels_dict = {'c0' : 'safe driving',
'c1' : 'texting - right',
'c2' : 'talking on the phone - right',
'c3' : 'texting - left',
'c4' : 'talking on the phone - left',
'c5' : 'operating the radio',
'c6' : 'drinking',
'c7' : 'reaching behind',
'c8' : 'hair and makeup',
'c9' : 'talking to passenger'}

In [7]:
test_images_files = list(test_img_dir.glob(r'**/*.jpg'))

In [10]:
test_df = pd.DataFrame(test_images_files).astype(str)

In [11]:
test_df.columns=['filepaths']

In [25]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    #preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)


In [26]:
#test_images.reset()
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepaths',
    y_col=None,
    target_size=(224, 224),
    rescale=1./255,
    color_mode='rgb',
    class_mode=None,
    batch_size=64,
    shuffle=False
)

Found 79726 validated image filenames.


In [14]:
models = list(model_dir.glob(r'**/*.hdf5'))
models

[PosixPath('../models/best_model_vgg16_dropout_1024.hdf5'),
 PosixPath('../models/best_model_vgg16_l2_1024.hdf5'),
 PosixPath('../models/best_model_vgg19_l2_4096_1024_adam_lr.hdf5'),
 PosixPath('../models/base_model_kfold.hdf5'),
 PosixPath('../models/best_model_EfficientNetB0_v2_augment.hdf5'),
 PosixPath('../models/best_model_vgg16_dropout_4096_512.hdf5'),
 PosixPath('../models/best_model_vgg_l2_4096_1024_adam_lr_IG.hdf5'),
 PosixPath('../models/best_model_vgg16_l2_4096_1024_adam_lr.hdf5'),
 PosixPath('../models/best_model_vgg16_l2_4096_1024.hdf5'),
 PosixPath('../models/base_model_gkfold.hdf5'),
 PosixPath('../models/best_model_EfficientNetB0_dense.hdf5'),
 PosixPath('../models/best_model.hdf5'),
 PosixPath('../models/best_model_EfficientNetB0_v2_base.hdf5'),
 PosixPath('../models/best_model_vgg16_dropout_1024_512_bs32.hdf5'),
 PosixPath('../models/best_model_vgg16.hdf5'),
 PosixPath('../models/best_model_vgg16_dropout_1024_512.hdf5'),
 PosixPath('../models/best_model_EfficientNetB0

In [15]:
len(models)

17

In [29]:
def kaggle_submission(test_images, loaded_model, filename):
    submission_csv = os.path.join(submissions_dir, f'{filename}.csv')
    my_file = Path(submission_csv)
    if my_file.is_file():
        print(f'already existing submission file for {submission_csv}')
        return

    print(f'Predicting Model {filename}')
    pred = loaded_model.predict(test_images, verbose=1)
    print(f'Creating a DataFrame of {filename} Predictions')
    pred_df = pd.DataFrame(data=pred, columns=list(class_labels_dict.keys()), index=(test_images.filenames))
    pred_df.index = pred_df.index.map(os.path.basename)
    pred_df.index.name='img'
    print(f'Preparing Submission file for {filename}')    
    pred_df.to_csv(submission_csv)
    # submission_csv = f'/content/drive/My\ Drive/Colab_Notebooks/state_farm/submissions/{filename}.csv' #needed for CLI. space in folder name
    print(f'Uploading to Kaggle {submission_csv}')
    !kaggle competitions submit -c state-farm-distracted-driver-detection -f {submission_csv}  -m "{filename} submission"

In [30]:
for model in models:
    filename = os.path.basename(model).replace('.hdf5', '')
    loaded_model = tf.keras.models.load_model(model)
    kaggle_submission(test_images, loaded_model, filename)
  # print(loaded_model.summary())

already existing submission file for ../submissions/best_model_vgg16_dropout_1024.csv
already existing submission file for ../submissions/best_model_vgg16_l2_1024.csv
already existing submission file for ../submissions/best_model_vgg19_l2_4096_1024_adam_lr.csv
Predicting Model base_model_kfold
1246/1246 [==============================] - 359s 288ms/step
Creating a DataFrame of base_model_kfold Predictions
Preparing Submission file for base_model_kfold
Uploading to Kaggle ../submissions/base_model_kfold.csv
zsh:1: command not found: kaggle
Predicting Model best_model_EfficientNetB0_v2_augment
1246/1246 [==============================] - 1003s 804ms/step
Creating a DataFrame of best_model_EfficientNetB0_v2_augment Predictions
Preparing Submission file for best_model_EfficientNetB0_v2_augment
Uploading to Kaggle ../submissions/best_model_EfficientNetB0_v2_augment.csv
zsh:1: command not found: kaggle
already existing submission file for ../submissions/best_model_vgg16_dropout_4096_512.csv


In [31]:
!pip install -q kaggle
!chmod 600 ~/.kaggle/kaggle.json


In [34]:
submissions = list(Path(submissions_dir).glob(r'**/*.csv'))
submissions

[PosixPath('../submissions/best_model_vgg_l2_4096_1024_adam_lr_IG.csv'),
 PosixPath('../submissions/best_model_vgg16_l2_1024.csv'),
 PosixPath('../submissions/best_model_vgg16.csv'),
 PosixPath('../submissions/basemodel.csv'),
 PosixPath('../submissions/EfficientNetB0_v2_augment_20epochs.csv'),
 PosixPath('../submissions/best_model_vgg16_dropout_1024_512.csv'),
 PosixPath('../submissions/best_model_vgg16_l2_4096_1024_adam_lr.csv'),
 PosixPath('../submissions/best_model_vgg16_l2_4096_1024.csv'),
 PosixPath('../submissions/best_model_EfficientNetB0_dense.csv'),
 PosixPath('../submissions/best_model_vgg16_dropout_4096_512.csv'),
 PosixPath('../submissions/best_model_EfficientNetB0_v2.csv'),
 PosixPath('../submissions/model_eval_scores.csv'),
 PosixPath('../submissions/best_model_vgg19_l2_4096_1024_adam_lr.csv'),
 PosixPath('../submissions/best_model.csv'),
 PosixPath('../submissions/base_model_kfold.csv'),
 PosixPath('../submissions/base_model_gkfold.csv'),
 PosixPath('../submissions/Effi

In [38]:
for submission in submissions:
    print(f'Kaggle Submission - {submission}')
    !kaggle competitions submit -c state-farm-distracted-driver-detection -f {submission}  -m "{submission} submission"

Kaggle Submission - ../submissions/best_model_vgg_l2_4096_1024_adam_lr_IG.csv
100%|███████████████████████████████████████| 10.8M/10.8M [00:16<00:00, 668kB/s]
Successfully submitted to State Farm Distracted Driver DetectionKaggle Submission - ../submissions/best_model_vgg16_l2_1024.csv
100%|███████████████████████████████████████| 10.5M/10.5M [00:16<00:00, 653kB/s]
Successfully submitted to State Farm Distracted Driver DetectionKaggle Submission - ../submissions/best_model_vgg16.csv
100%|███████████████████████████████████████| 10.6M/10.6M [00:16<00:00, 652kB/s]
Successfully submitted to State Farm Distracted Driver DetectionKaggle Submission - ../submissions/basemodel.csv
100%|███████████████████████████████████████| 11.0M/11.0M [00:18<00:00, 613kB/s]
Successfully submitted to State Farm Distracted Driver DetectionKaggle Submission - ../submissions/EfficientNetB0_v2_augment_20epochs.csv
100%|███████████████████████████████████████| 10.5M/10.5M [00:17<00:00, 645kB/s]
Successfully submi